In [4]:
#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
#MY_HOME_ABS_PATH = "/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling"
MY_HOME_ABS_PATH = "/root/co2-flux-hourly-gpp-modeling"

In [5]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
  from google.colab import drive
  drive.mount('/content/drive/')
else:
  IN_COLLAB = False

In [6]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pandas as pd
import numpy as np
import sys
import joblib
from io import BytesIO
from sklearn.model_selection import StratifiedKFold

# Load locale custome modules
os.chdir(MY_HOME_ABS_PATH)
if IN_COLLAB:
    sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
    sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Definitions

In [7]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'
preproc_objects_dir = root_dir + os.sep + 'code' + os.sep + 'src' + os.sep + 'preprocessing' + os.sep + 'preproc_objects'

if IN_COLLAB:
    raw_data_dir = "/content/drive/MyDrive/W210/Data/half_hourly_data"

# input files
site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'
monthly_data_filename = data_dir + os.sep + "monthly-mvp.csv"
split_dict_filename = preproc_objects_dir + os.sep + "stratified_splits_k5.joblib"

# File
container = "all-sites-data"
ext = "parquet"
ver = "mvp"
model = "rfr"
tag = "raw"
blob_name_base = f"{model}-full_2010_2015_v_{ver}"
blob_name = f"{blob_name_base}_{tag}.{ext}"
train_blob_name = f"{model}-full_2010_2015-train-v-{ver}.{ext}"
val_blob_name = f"{model}-full_2010_2015-val-v-{ver}.{ext}"
test_blob_name = f"{model}-full_2010_2015-test-v-{ver}.{ext}"

In [8]:
# Define features and target variables of the data pipelines
target_variable = 'GPP_NT_VUT_REF'
hourly_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7']
metadata_features = ['site_id', 'filename', 'koppen_sub', 'koppen_main',
                     'c3c4', 'c4_percent', 'monthly_data_available']

# Define the features to use in KNN imputer, only using real values as cat are same per site
imp_exclude_cols = ['date', 'datetime', 'year', 'month', 'hour', 'day', 'minute', 'site_id', 'IGBP']
imp_cols = [x for x in hourly_features + ['GPP_NT_VUT_REF'] if x not in imp_exclude_cols]

### Load One Site

In [9]:
# Prepare hourly site df
site_id = 'AU-ASM'
filename = f'data_full_half_hourly_raw_v0_1_{site_id}.csv'
local_filename = tmp_dir + os.sep + filename
site_df = pd.read_csv(local_filename)

# Format columns
site_df['datetime'] = pd.to_datetime(site_df['datetime'])
site_df['date'] = pd.to_datetime(site_df['date'])
site_df['minute'] = site_df['datetime'].dt.minute
site_df['site_id'] = site_id

# Move from HH to H level
site_df = site_df.loc[site_df['datetime'].dt.minute == 0, ].copy()
site_df.drop('minute', axis=1, inplace=True)

In [10]:
site_df.head(2)

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,201009030000,201009030030,17.05000,0,20.06300,0.00000,0.00000,0,0.00000,344.00000,0.00000,369.07700,1.36100,0,5.68100,0.00000,0.00000,0.00000,94.25000,0.00000,94.27700,-54.00000,NaN,-40.00000,0.00000,2.00000,0.00000,2.67961,-7.00000,0.00000,-9.35371,1.92074,0.00000,1.92074,0.00000,-0.88585,0.00000,-0.88585,0.00000,1.03489,0.75614,1.03198,0.78076,2010-09-03 00:00:00,2010,9,3,0,AU-ASM,2010-09-03,1.92074,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM
2,201009030100,201009030130,16.68000,0,20.10000,0.00000,0.00000,0,0.00000,353.00000,0.00000,370.18200,1.33000,0,5.62200,0.00000,0.00000,0.00000,94.29000,0.00000,94.22600,-43.00000,NaN,-35.00000,0.00000,4.00000,1.00000,5.35922,-11.04760,1.00000,-14.76230,1.59790,1.00000,1.33708,1.00000,-0.58022,0.00000,-0.58022,0.00000,1.01768,0.74104,1.01503,0.76282,2010-09-03 01:00:00,2010,9,3,1,AU-ASM,2010-09-03,1.59790,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM


## Engineer Features

#### MSC Features

In [17]:
# MSC Features
def engineer_msc_features(df, input_features):
    seasons = {
            "Winter": [12, 1, 2],
            "Spring": [3, 4, 5],
            "Summer": [6, 7, 8],
            "Fall": [9, 10, 11]
        }

    # Function to map months to the corresponding season
    def month_to_season(month):
        for season, months in seasons.items():
            if month in months:
                return season
        return None
    df['season'] = df.month.map(month_to_season)

    # Loop through input features
    for feature in input_features:
        # Get MSC features
        mean_cycles = {}
        for season, months in seasons.items():
            mean_cycles[season] = df[df['season'] == season][feature].mean()

        # Get Amplitude, Min features
        amplitude_msc = max(mean_cycles.values()) - min(mean_cycles.values())
        min_msc = min(mean_cycles.values())

        # Merge into DF
        for season, mean_value in mean_cycles.items():
            df.loc[df['season'] == season, f"{feature}_szn_mean"] = mean_value
        df[f"{feature}_amp_msc"] = amplitude_msc
        df[f"{feature}_min_msc"] = min_msc

    # Remove season col
    df.drop(columns=['season'], inplace=True)

    return df

msc_features = ['TA_ERA', 'SW_IN_ERA', 'P_ERA', 'EVI', 'NDVI', 'NIRv', 'b4']
test = engineer_msc_features(site_df, msc_features)
test.head(2)

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id,TA_ERA_szn_mean,TA_ERA_amp_msc,TA_ERA_min_msc,SW_IN_ERA_szn_mean,SW_IN_ERA_amp_msc,SW_IN_ERA_min_msc,P_ERA_szn_mean,P_ERA_amp_msc,P_ERA_min_msc,EVI_szn_mean,EVI_amp_msc,EVI_min_msc,NDVI_szn_mean,NDVI_amp_msc,NDVI_min_msc,NIRv_szn_mean,NIRv_amp_msc,NIRv_min_msc,b4_szn_mean,b4_amp_msc,b4_min_msc
0,201009030000,201009030030,17.05000,0,20.06300,0.00000,0.00000,0,0.00000,344.00000,0.00000,369.07700,1.36100,0,5.68100,0.00000,0.00000,0.00000,94.25000,0.00000,94.27700,-54.00000,NaN,-40.00000,0.00000,2.00000,0.00000,2.67961,-7.00000,0.00000,-9.35371,1.92074,0.00000,1.92074,0.00000,-0.88585,0.00000,-0.88585,0.00000,1.03489,0.75614,1.03198,0.78076,2010-09-03 00:00:00,2010,9,3,0,AU-ASM,2010-09-03,1.92074,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468
2,201009030100,201009030130,16.68000,0,20.10000,0.00000,0.00000,0,0.00000,353.00000,0.00000,370.18200,1.33000,0,5.62200,0.00000,0.00000,0.00000,94.29000,0.00000,94.22600,-43.00000,NaN,-35.00000,0.00000,4.00000,1.00000,5.35922,-11.04760,1.00000,-14.76230,1.59790,1.00000,1.33708,1.00000,-0.58022,0.00000,-0.58022,0.00000,1.01768,0.74104,1.01503,0.76282,2010-09-03 01:00:00,2010,9,3,1,AU-ASM,2010-09-03,1.59790,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468


#### Precipitation Rolling sums

In [18]:
def engineer_prcp_running_sums(df, precip_col='P_ERA'):
    # Set the datetime column as index
    df = df.set_index('datetime')

    # Calculate the running sum of the last week's precipitation (168 hours)
    df[f"prcp_week_sum"] = df[precip_col].rolling(window='168H').sum()
    df[f"prcp_month_sum"] = df[precip_col].rolling(window='720H').sum()

    # Reset the index
    df.reset_index(inplace=True)

    return df

test = engineer_prcp_running_sums(site_df, 'P_ERA')
test.head(20)

,datetime,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id,TA_ERA_szn_mean,TA_ERA_amp_msc,TA_ERA_min_msc,SW_IN_ERA_szn_mean,SW_IN_ERA_amp_msc,SW_IN_ERA_min_msc,P_ERA_szn_mean,P_ERA_amp_msc,P_ERA_min_msc,EVI_szn_mean,EVI_amp_msc,EVI_min_msc,NDVI_szn_mean,NDVI_amp_msc,NDVI_min_msc,NIRv_szn_mean,NIRv_amp_msc,NIRv_min_msc,b4_szn_mean,b4_amp_msc,b4_min_msc,prcp_week_sum,prcp_month_sum
0,2010-09-03 00:00:00,201009030000,201009030030,17.05000,0,20.06300,0.00000,0.00000,0,0.00000,344.00000,0.00000,369.07700,1.36100,0,5.68100,0.00000,0.00000,0.00000,94.25000,0.00000,94.27700,-54.00000,NaN,-40.00000,0.00000,2.00000,0.00000,2.67961,-7.00000,0.00000,-9.35371,1.92074,0.00000,1.92074,0.00000,-0.88585,0.00000,-0.88585,0.00000,1.03489,0.75614,1.03198,0.78076,2010,9,3,0,AU-ASM,2010-09-03,1.92074,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468,0.00000,0.00000
1,2010-09-03 01:00:00,201009030100,201009030130,16.68000,0,20.10000,0.00000,0.00000,0,0.00000,353.00000,0.00000,370.18200,1.33000,0,5.62200,0.00000,0.00000,0.00000,94.29000,0.00000,94.22600,-43.00000,NaN,-35.00000,0.00000,4.00000,1.00000,5.35922,-11.04760,1.00000,-14.76230,1.59790,1.00000,1.33708,1.00000,-0.58022,0.00000,-0.58022,0.00000,1.01768,0.74104,1.01503,0.76282,2010,9,3,1,AU-ASM,2010-09-03,1.59790,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468,0.00000,0.00000
2,2010-09-03 02:00:00,201009030200,201009030230,18.17000,0,20.05100,0.00000,0.00000,0,0.00000,386.00000,0.00000,370.18200,2.92100,0,5.56300,0.00000,0.00000,0.00000,93.77000,0.00000,94.19700,-17.00000,NaN,-23.00000,0.00000,7.25000,1.00000,9.71359,-12.87500,1.00000,-17.20410,0.93635,1.00000,-0.73019,1.00000,0.15116,0.00000,0.15116,0.00000,1.08751,0.80262,1.08376,0.83630,2010,9,3,2,AU-ASM,2010-09-03,0.93635,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468,0.00000,0.00000
3,2010-09-03 03:00:00,201009030300,201009030330,18.03000,0,20.00200,0.00000,0.00000,0,0.00000,390.00000,0.00000,370.18200,2.68900,0,5.50300,0.00000,0.00000,0.00000,93.58000,0.00000,94.16800,-12.00000,NaN,-12.00000,0.00000,6.84211,1.00000,9.16709,-13.05260,1.00000,-17.44150,0.93635,1.00000,-0.73019,1.00000,0.14455,0.00000,0.14455,0.00000,1.08089,0.79675,1.07725,0.82926,2010,9,3,3,AU-ASM,2010-09-03,0.93635,NaN,0.15840,0.33739,0.06299,0.09250,0.18670,0.03400,0.05720,0.24940,0.24470,0.19490,SAV,Arid,AU-ASM,24.78207,13.98399,14.70853,293.90267,110.90708,195.09566,0.01158,0.04404,0.00093,0.12416,0.02320,0.12303,0.25139,0.06207,0.25139,0.05097,0.01150,0.04723,0.06689,0.01601,0.05468,0.00000,0.00000
4,2010-09-03 04:00:00,201009030400,201009030430,17.92000,0,19.83900,0.00000,0.00000,0,0.00000,387.00000,0.00000,374.89200,2.67000,0,5.23400,0.00000,0.00000,0.00000,93.88000,0.00000,94.15200,-15.00000,NaN,-15.00000,0.00000,2.00000,0.00000,2.67961,-8.00000,0.00000,-10.69000,1.03457,1.00000,-0.47977,1.00000,0

#### Hemisphere Feature (on meta df)

In [20]:
site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'
site_meta = pd.read_csv(site_metadata_filename)
print(len(site_meta))
site_meta.head(2)

286


,site_id,dataset,start_year,end_year,file,is_dup,IGBP,elevation,lat,long,site_name,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename,size,country,record_count,site_IGBP,site_koppen,start_time,end_time,recorded_day_count,recorded_day_percentage,monthly_data_available
0,AR-SLu,FLUXNET,2009,2011,FLX_AR-SLu_FLUXNET2015_FULLSET_MM_2009-2011_1-...,False,MF,NaN,-33.46480,-66.45980,San Luis,7,2,BSk,Arid,C3,67.08000,data_full_half_hourly_raw_v0_1_AR-SLu.csv,9785735.00000,AR,22128.00000,MF,Arid,2009-12-21 00:00:00,2011-03-26 23:30:00,461.00000,1.00000,Yes
1,AR-Vir,FLUXNET,2009,2012,FLX_AR-Vir_FLUXNET2015_FULLSET_MM_2009-2012_1-...,False,ENF,NaN,-28.23950,-56.18860,Virasoro,14,3,Cfa,Temperate,C3,8.75000,data_full_half_hourly_raw_v0_1_AR-Vir.csv,14564015.00000,AR,33984.00000,ENF,Temperate,2010-02-13 00:00:00,2012-06-13 23:30:00,708.00000,0.83099,Yes


In [21]:
# Simple North vs. South hemisphere
def get_hemisphere_NS(latitude):
    if latitude >= 0:
        return 'North'
    else:
        return 'South'

site_meta['hemisphere_NS'] = site_meta['lat'].apply(get_hemisphere_NS)

In [23]:
# 5 hemisphere bands
def get_latitude_band(latitude):
    if -90 <= latitude < -54:
        return 'lat_band_1'
    elif -54 <= latitude < -18:
        return 'lat_band_2'
    elif -18 <= latitude < 18:
        return 'lat_band_3'
    elif 18 <= latitude < 54:
        return 'lat_band_4'
    elif 54 <= latitude <= 90:
        return 'lat_band_5'
    else:
        return 'Invalid latitude'

# Assuming your DataFrame is named 'data' and has a 'latitude' column
# Create a new column called 'latitude_band' based on the 'latitude' column
site_meta['lat_band'] = site_meta['lat'].apply(get_latitude_band)

print(len(site_meta))

286


In [24]:
# Save out to overwrite site meta with new features
#site_meta.to_csv(site_metadata_filename, index=False)